# Implicit solution to ODEs

## Root finding: Newton-Raphson method

**Literature: ** Numerical Recipes, Chapter 9.0 and 9.4 

Root-finding is required in many applications of computational physics. It is at the core of the fundamental task to solve an equation numerically. Any equation with a right-hand and left-hand side can be rearranged to 
$$ f(x) = 0.$$
If the there is only one independent variable we have just a one-dimensional problem. The Newton-Raphson method is a robust and easy method to find the root of a function. It has very fast convergence if there is a good initial guess, but it has - like other methods - several pathologies that one needs to be aware off. One of its advantages is that it does generalize to multiple dimensions in a straight-forward way. 

### Mathematical idea of the method

The method starts with a good initial guess $x_{ini}$ of the root. Evaluating the function and the derivative for the initial guess provides a correction $\delta$ toward the actual root $x'$.

![nr-method](../images/NR_scheme.jpg)

We can then repeat this improvement until a termination criterion certifies that the solution is good enough. 

###Algorithm
Once we have the basic idea and mathematical formulation of the method in place, and before we actually start the coding it often is useful to write our strategy down in the form of an algorithm.

![nr-algorithm](../images/NR_algorithm.jpg)


###Termination condition

The termination conditions determine if a solution is good enough to
be accepted. Really, we are checking on the precision with which the
solution is satisfying the discretized equation. The termination
conditions for the NR method may include the following:

- $\delta < \delta_{lim}$: corrections are smaller than some limit
- the function evaluation is smaller than some limit, i.e. close to zero: $f(x_n)< \epsilon_{fmax}$ 

### Pathological cases

This termination condition can check on a number of things, and the
particular choices may depend on the particular problem. Before
writing down the termination conditions it makes sense to anticipate
pathological cases, i.e. situations in which this method will fail:

- higher-order terms are important, and initial guess is not good enough $\rightarrow$ the method will send you off to infinity
- non-convergent cycles: there could be a case, for example, when table look-up is involved in calculating the derivative, that correction are bouncing the guess from 
one side of the root to the other indefinitely without ever coming closer
- there could be multiple roots, and the one the NR method is finding is only a local root selected by the choice of initial guess.

Note that all methods have pathologies, or in other words, have their
particular weaknesses and strength. Selecting the most appropriate
method for a particular problem requires you to consider the
advantages and disadvantages of different methods.


The following remedies can be added to the termination conditions to accomodate the pathological cases:

- corrections are monotonically decreasing
- limit maximum number of iterations 
- restart iterations with different initial guess



### The Numerical Recipes Fortran code
Here is a Fortran 77 code that implements the one-dimensional 
Newton-Raphson method. Let's analyze this code. We will learn 
a little bit about legacy Fortran code on the side

```
      FUNCTION rtnewt(funcd,x1,x2,xacc)
	  INTEGER JMAX	
	  REAL rtnewt,x1,x2,xacc
	  EXTERNAL funcd
	  PARAMETER (JMAX=20)
      INTEGER j
	  REAL df,dx,f
	  rtnewt=.5*(x1+x2)
      do 11 j=1,JMAX
  	    call funcd(rtnewt,f,df)
  	    dx=f/df
        rtnewt=rtnewt-dx
        if((x1-rtnewt)*(rtnewt-x2).lt.0.)pause
     *         'rtnewt jumped out of brackets'
        if(abs(dx).lt.xacc) return
11    continue
      pause 'rtnewt exceeded maximum iterations'
      END
C  (C) Copr. 1986-92 Numerical Recipes Software
```


**Homework:**

1. Implement the Newton-Raphson method for a scalar function in your CompPhys module.
2. Create a test notebook that verifies its proper working and demonstrates its use for some simple examples where you know the answer.

## Implicit ODE solution 

The Newton-Raphson method can be used as well for an alternative
approach to integrating the ODE that appeared in the sky diver
problem. So far we have used an _explicit_ forward integration which
involves evaluating the RHS of the ODE using the know velocity
$ v_n $ at the present time to get the value of the
velocity 
at the next time step $ n+1 $. In cases where
we are interested in the longer-term behaviour, and where we are
required to take larger time steps this may not work very well. In
that case we want to evaluate the RHS for $ v(n+1) $, and we have
an _implicit_ solution scheme. Especially for large coupled systems of
ODEs implicit schemes ofetn provide the numerical stability to find a
solution at all. The implicit solution involves an iterative approach,
and we can use again the Newton-Raphson method:
![nr-method](../images/NR_for_ODEs.jpg)

As you can see the basic strategy is to put everything on one side,
give it a name $ G(v_{n+1})$, set this function to zero and
solve it by Newton-Raphson. The beauty is that this idea can be easily
expanded to systems of ODEs with many variables. In fact this is the
method that is used for large reaction rate networks involving
thousands of isotopes, and in most (all?) research grade stellar
evolution codes to solve the coupled partial differential equations of
mass, momentum and energy conservation.

**Homework:**

1. Implement a method for solving a 1D ODE implicitly using the Newton-Raphson method.
2. Create a notebook that solves the sky diver problem implicitely. Your notebook should demonstrate how the implicit scheme is different compared to the explicit scheme. It should also contain elements of verification.

## Multi-dimensional problems

###Working with vectors

This would allow us to extend 1D methods to 2D. For example, we would
like to add to the sky diver problem a horizontal wind with a vertical
profile and calculate how far from the drop off location the sky diver
will land.

The most straight-forward representation of a vector would be an array
with two entries. This could hold a velocity vector with a horizontal
and vertical component of the velocity. A list with two entries could
hold function that describe the RHS of a system of two ODEs that
describe the horizontal and vertical velocity.

It would be nice to animate our equation of motion results. There is a package that supports this called _visual python_. The package can be installed in your past machine and it is called `python-visual`, see as well <http://www.vpython.org>. Using this approach is covered step-by-step in this freely downloadable ebook [Introduction to Computational Physics](http://www.lulu.com/shop/david-roundy/introduction-to-computational-physics/ebook/product-17437845.html). It is generally useful and covers additional aspects of this course from an alternative perspective.

**Homework:**

1. install vpython in your past machine
2. explore animation of object using this software package
3. determine scope and limitations


### Sky diver in 2D

**Homework:**

The sky diver will be exposed to a horizontal wind that decreases quadratically. Given the horizontal velocity at the jump-off height write a program that will determine the touch-down location.

Hints:

1. The vertical profile of the horizontal wind velocity has a natural boundary condition at the ground.
2. The vertical and horizontal wind drag may not be the same. Make reasonable assumoptions, and test how sensitive your results are on your assumptions.